In [1]:
import sys
import pickle
from pprint import pprint 
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from data.make_casting_graph import oneway_to_bidirected_graph
from scipy.sparse import csc_matrix
import time
from pagerank import pagerank
from sklearn.preprocessing import normalize
from pyvis.network import Network
from pagerank import pagerank

In [2]:
# create idx to num comments
with open('./data/ratings.csv', encoding='utf-8') as f:
    docs = [line.strip().split(',') for line in f.readlines()[1:]]
    _idx2numcomments = {movie_idx:int(num) for num, movie_idx in docs}

In [3]:
# pre defined casting weight graph
with open('./data/casting_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

In [4]:
# create idx to actor name function
with open('./data/actors.csv', encoding='utf-8') as f:
    next(f)
    docs = [line.split(',') for line in f.readlines()[1:]]
    # English name if exist else Korean name
    _idx2actor = {doc[0]:doc[1] for doc in docs}

In [5]:
with open('./data/movies.csv', encoding='utf-8') as f:
    next(f)
    docs = [line.split(',') for line in f.readlines()[1:]]
    _idx2movie = {doc[0]:doc[1] for doc in docs if len(docs)}

In [6]:
idx2movie = lambda idx: _idx2movie.get(idx, 'Unknown')
idx2actor = lambda idx: _idx2actor.get(idx, 'Unknown')
idx2numcomments = lambda idx: _idx2numcomments.get(idx,0)

In [7]:
g = oneway_to_bidirected_graph(graph)

In [8]:
for movie in sorted(_idx2numcomments.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(idx2movie(movie[0]), movie[1])

기생충 40
극한직업 15
마약왕 15
인터스텔라 14
어벤져스: 엔드게임 12
걸캅스 12
마녀 12
택시운전사 11
배심원들 11
신과함께-죄와 벌 11


In [9]:
def make_graph(casting_csv_path):
    # load file
    with open(casting_csv_path, encoding='utf-8') as f:
        next(f)
        graph = {line.split('\t')[0]:line.split('\t')[1].strip().split() for line in f if len(line.split('\t'))==2}
    
    # weighting
    # casting order (n-i)^2/ sum (i^2 for i = 1 to n)
    def weight(casting_order):
        if not casting_order:
            return {}
        n = len(casting_order)
        weights = [(n-i) ** 2 for i in range(n)]
        sum_ = sum(weights)
        return {actor:w/sum_ for actor, w in zip(casting_order, weights)}
    
    graph = {movie:weight(actors) for movie, actors in graph.items() if actors}
    return graph

def oneway_to_bidirected_graph(graph):
    """Input: graph[movie][actor] = weight graph"""
    # bi-directed graph
    # graph has only one-way link: movie -> actor
    actor_weight_sum = {}

    # cumulate actor weights
    for movie, actors in graph.items():
        for actor, weight in actors.items():
            actor_weight_sum[actor] = actor_weight_sum.get(actor, 0) + weight

    # make bi-directed graph
    from collections import defaultdict
    g = defaultdict(lambda: {})
    for movie, actors in graph.items():
        g['movie {}'.format(movie)] = {'actor {}'.format(a):w for a,w in actors.items()}
        for actor, weight in actors.items():
            g['actor {}'.format(actor)]['movie {}'.format(movie)] = weight / actor_weight_sum[actor]

    g = dict(g)
    return g

def main():
    casting_csv_path = './data/casting.txt'
    graph_path = './data/casting_graph.pkl'

    graph = make_graph(casting_csv_path)

    import pickle
    with open(graph_path, 'wb') as f:
        pickle.dump(graph, f)

if __name__ == '__main__':
    main()

In [10]:
import time

In [11]:
def pagerank(G, bias=None, df=0.15,
             max_iter=50, converge_error=0.001,verbose=0):
    """
    Arguments
    ---------
    G: Inbound graph, dict of dict
        G[to_node][from_node] = weight (float)
    df: damping factor, float. default 0.15
    """
    
    A, nodes_dict = _normalize(G)
    N = len(nodes_dict) # number of nodes
    sr = 1 - df # survival rate (1 -  damping factor)
    ir = 1 / N # initial rank

    # Initialization
    rank_dict = {n:ir for n in nodes_dict}

    # Initialization of bias
    if not bias:
        bias = {node:ir for node in nodes_dict}

    # Iteration
    for _iter in range(1, max_iter + 1):
        rank_dict_new = {}

        # t: to node, f: from node, w: weight
        for t in nodes_dict:
            f_dict = A.get(t, {})
            rank_dict_t = sum((w*rank_dict[f] for f, w in f_dict.items())) if f_dict else 0
            rank_dict_t = sr * rank_dict_t + df * bias.get(t, 0)
            rank_dict_new[t] = rank_dict_t

        # convergence check
        diff = sum((abs(rank_dict[n] - rank_dict_new[n]) for n in nodes_dict))
        if diff < converge_error:
            if verbose:
                print('Early stopped at iter = {}'.format(_iter))
            break

        if verbose:
            sum_ = sum(rank_dict_new.values())
            print('Iteration = {}, diff = {}, sum = {}'.format(_iter, diff, sum_))

        rank_dict = rank_dict_new

    return rank_dict


def _normalize(G):
    """It returns outbound normalized graph
    Arguments
    ---------
    G: inbound graph dict of dict
    """
    # Sum of outbound weight
    # t: to node, f: from node, w: weight
    W_sum = {}    
    for t, f_dict in G.items():
        for f, w in f_dict.items():
            W_sum[f] = W_sum.get(f, 0) + w
    A = {t:{f:w/W_sum[f] for f,w in f_dict.items()} for t, f_dict in G.items()}    
    nodes_dict = set(G.keys())
    nodes_dict.update(W_sum)
    return A, nodes_dict

## [과제1] 성능비교

### 　>> Dictionary

In [12]:
bias = {node:(idx2numcomments(node.split()[1]) if node[0] == 'm' else 0) for node in g}
_sum = sum(bias.values())
bias = {node:b / _sum for node, b in bias.items()}

starttime = time.time()
rank_dict = pagerank(g,
               bias = bias,
               df = 0.15,
               max_iter = 30,
               converge_error = 0.001,
               verbose = 1)
print('computation time : {0}s'.format(round(time.time()-starttime, 3)))

Iteration = 1, diff = 0.6745935594038667, sum = 1.000000000000005
Iteration = 2, diff = 0.5133755765513068, sum = 1.0000000000000064
Iteration = 3, diff = 0.4070843471025288, sum = 1.0000000000000073
Iteration = 4, diff = 0.3288114569044879, sum = 1.000000000000001
Iteration = 5, diff = 0.269000062616973, sum = 1.0000000000000044
Iteration = 6, diff = 0.2217292304456657, sum = 0.9999999999999902
Iteration = 7, diff = 0.18372765496993057, sum = 0.9999999999999921
Iteration = 8, diff = 0.15290648077655614, sum = 1.0000000000000033
Iteration = 9, diff = 0.12756391624362184, sum = 0.9999999999999933
Iteration = 10, diff = 0.10676563571706411, sum = 0.9999999999999923
Iteration = 11, diff = 0.08947335545631485, sum = 1.0000000000000075
Iteration = 12, diff = 0.07517014319662849, sum = 1.0000000000000107
Iteration = 13, diff = 0.0631852881114481, sum = 0.9999999999999923
Iteration = 14, diff = 0.05320609097840659, sum = 0.9999999999999906
Iteration = 15, diff = 0.04483047792706704, sum = 1.0

In [13]:
import numpy as np
from scipy.sparse import csc_matrix

In [14]:
nodes = set(g.keys())
idx2node = list(sorted(nodes))
node2idx = {node:idx for idx, node in enumerate(idx2node)}

bias = np.asarray([b for node, b in sorted(bias.items(), key = lambda tp: node2idx[tp[0]])])
print(bias.shape)

rows = []
cols = []
data = []

for from_node, to_dict in g.items():
    from_idx = node2idx[from_node]
    for to_node, weight in to_dict.items():
        to_idx = node2idx[to_node]
        rows.append(from_idx)
        cols.append(to_idx)
        data.append(weight)

A = csc_matrix((data, (rows, cols)))
print(A.shape)

(6154,)
(6154, 6154)


### 　>> Numpy

In [15]:
starttime2 = time.time()
max_iter = 30
df = 0.85
ir = 1 / A.shape[0]
rank = np.asarray([ir] * A.shape[0])
for n_iter in range(1, max_iter + 1):
    rank_new = A.dot(rank)
    rank_new = normalize(rank_new.reshape(1, -1), norm = 'l1').reshape(-1)
    rank_new = df * rank_new + (1 - df) * bias
    diff = abs(rank - rank_new).sum()
    rank = rank_new
    print('iter {} : diff = {}'.format(n_iter, diff))
print('computation time : {0}s'.format(round(time.time()-starttime2, 3)))

iter 1 : diff = 0.1685245368865779
iter 2 : diff = 0.123534416788289
iter 3 : diff = 0.11717242074154521
iter 4 : diff = 0.08676250638774644
iter 5 : diff = 0.08106650827175174
iter 6 : diff = 0.06044614044638538
iter 7 : diff = 0.05589952786903922
iter 8 : diff = 0.04188475454126574
iter 9 : diff = 0.038452782327255894
iter 10 : diff = 0.0289095171904886
iter 11 : diff = 0.026405522194198443
iter 12 : diff = 0.01994486388644759
iter 13 : diff = 0.01811137289916391
iter 14 : diff = 0.013753287448751986
iter 15 : diff = 0.012408911428306675
iter 16 : diff = 0.009469243738374537
iter 17 : diff = 0.008494000468005527
iter 18 : diff = 0.006511648928942716
iter 19 : diff = 0.005809774127703195
iter 20 : diff = 0.004473307017566352
iter 21 : diff = 0.0039712967053357525
iter 22 : diff = 0.0030704578506105173
iter 23 : diff = 0.0027152845982687866
iter 24 : diff = 0.002106149459828414
iter 25 : diff = 0.0018577039374234091
iter 26 : diff = 0.0014438021951808503
iter 27 : diff = 0.001270456142

## [과제2] 영화 Top 10

### 　>> Dictionary

In [16]:
movie_rank = {node:rank for node, rank in rank_dict.items() if node[0] == 'm'}

for movie_number, rating_rank in sorted(movie_rank.items(), key=lambda x:-x[1])[:10]:
    movie_idx = movie_number.split()[1]
    print(movie_number[6:],',',idx2movie(movie_idx),',',rating_rank)

161967 , 기생충 , 0.0032033878121671224
167651 , 극한직업 , 0.0014303471787626468
175322 , 마녀 , 0.0011565783119412997
156464 , 보헤미안 랩소디 , 0.0011527961465662747
130966 , 부산행 , 0.001098819013448319
177483 , 배심원들 , 0.0009469824923736168
174065 , 걸캅스 , 0.0009354687095915042
37886 , 클레멘타인 , 0.000918249213245038
154449 , 리틀 포레스트 , 0.00091821747845663
163788 , 알라딘 , 0.0007997936563664337


### 　>> Numpy

In [17]:
rank_ = {idx2node[idx]:value for idx, value in enumerate(rank)}
movie_rank2 = {node:value for node, value in rank_.items() if 'movie' in node}

for movie_number, rating_rank in sorted(movie_rank2.items(), key=lambda x:-x[1])[:10]:
    movie_idx = movie_number.split()[1]
    print(movie_number[6:],',',idx2movie(movie_idx),',',rating_rank)

161967 , 기생충 , 0.0015437432925532173
156464 , 보헤미안 랩소디 , 0.0010864984266341052
175322 , 마녀 , 0.0008946794759721638
174065 , 걸캅스 , 0.0008564445054703045
167651 , 극한직업 , 0.0007648489380972874
37886 , 클레멘타인 , 0.000728929546919159
157297 , 마약왕 , 0.0007133104346250872
71509 , 아저씨 , 0.0006938076365826392
136900 , 어벤져스: 엔드게임 , 0.0006567566198412949
163788 , 알라딘 , 0.000638759850450271


## [과제3] 영화 Top 노드 시각화

### 　>> Dictionary

In [18]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
got_net.barnes_hut()

for movie_number,_ in sorted(movie_rank.items(), key=lambda x:-x[1])[:10]:
    movie_idx = movie_number.split()[1] #top10 영화 Id 
    #top10 영화 이름으로 노드 생성
    got_net.add_node(idx2movie(movie_idx), label='Top10영화: ' + idx2movie(movie_idx)) 
    #top10 영화에 출연하는 배우들로 노드를 생성하고 edge 연결
    for actorNumber in g[movie_number]:   
        got_net.add_node(actorNumber, label='배우이름: ' + _idx2actor[actorNumber[6:]])
        got_net.add_edge(idx2movie(movie_idx), actorNumber)
        #top10 영화에 출연하는 배우들이 출연하는 다른 영화들의 노드를 생성하고 edge 연결
        for movieName in g.get(actorNumber):
            #top10에 존재하는 영화들의 중복을 제거
            if not movie_idx in movieName:
                got_net.add_node(movieName, label='영화: ' + _idx2movie[movieName[6:]])
                got_net.add_edge(actorNumber, movieName)

neighbor_map = got_net.get_adj_list()

got_net.show("DictionaryGraph.html")

### 　>> Numpy

In [19]:
got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
got_net.barnes_hut()

for movie_number,_ in sorted(movie_rank2.items(), key=lambda x:-x[1])[:10]:
    movie_idx = movie_number.split()[1] #top10 영화 Id 
    #top10 영화 이름으로 노드 생성
    got_net.add_node(idx2movie(movie_idx), label='Top10영화: '+idx2movie(movie_idx)) 
    #top10 영화에 출연하는 배우들로 노드를 생성하고 edge 연결
    for actorNumber in g[movie_number]:   
        got_net.add_node(actorNumber, label='배우이름: ' + _idx2actor[actorNumber[6:]])
        got_net.add_edge(idx2movie(movie_idx), actorNumber)
        #top10 영화에 출연하는 배우들이 출연하는 다른 영화들의 노드를 생성하고 edge 연결
        for movieName in g.get(actorNumber):
            #top10에 존재하는 영화들의 중복을 제거
            if not movie_idx in movieName:
                got_net.add_node(movieName, label='영화: ' + _idx2movie[movieName[6:]])
                got_net.add_edge(actorNumber, movieName)

neighbor_map = got_net.get_adj_list()

got_net.show("NumpyGraph.html")